# Anonimização por Perturbação

## Importações e definições de variáveis/funções globais

In [33]:
import numpy as np
import pandas as pd
import itertools as it
from datetime import datetime as dt
import re
import unicodedata
import random
from IPython.display import clear_output

filename        = "Dataset_Covid_CE.csv"
filename_out    = "Dataset_Covid_CE_Anon_Per.csv"

df_municipios = pd.read_csv('ce-regions-format.csv')

## Leitura e limpeza do dataset inicial
Fazemos apenas uma limpeza básica para não modificar muito os resultados do algoritmo.

In [34]:
# definição dos semi-identificadores
semi_ids = [
    "municipioCaso",
    "dataNascimento",
]

# definição dos atributos sensíveis
sensitive = [
    "comorbidadeCardiovascularSivep",
    "comorbidadeDiabetesSivep",
]

# definição dos tipos
dtype = {
    "municipioCaso"                  : "str",
    "dataNascimento"                 : "str",
    "comorbidadeCardiovascularSivep" : "str",
    "comorbidadeDiabetesSivep"       : "str",
}

# definição das colunas de data
dates = [
    "dataNascimento",
]
date_parser = lambda x: pd.to_datetime(x, format="%Y-%m-%d", errors = 'coerce')

# pegar apenas os atributos desejados do dataset
# df = pd.read_csv(filename, usecols=semi_ids + sensitive, dtype=dtype)
df = pd.read_csv(filename, usecols=semi_ids + sensitive, dtype=dtype, parse_dates=dates, date_parser=date_parser)
# df = pd.read_csv(filename, nrows=500000, usecols=semi_ids + sensitive, dtype=dtype)
df = df.dropna(how="any", subset=semi_ids) # remover registros com semi_ids nulo
df['municipioCaso'] = df['municipioCaso'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8') # remove acentuação
df


,municipioCaso,dataNascimento,comorbidadeCardiovascularSivep,comorbidadeDiabetesSivep
19,SOBRAL,2003-08-14,NaN,NaN
20,PACAJUS,1983-11-07,NaN,NaN
21,HORIZONTE,1982-01-14,NaN,NaN
25,FORTALEZA,1992-03-12,NaN,NaN
28,CAUCAIA,1970-03-06,NaN,NaN
...,...,...,...,...
1266272,FORTALEZA,1982-12-08,NaN,NaN
1266273,FORTALEZA,1980-12-02,NaN,NaN
1266274,FORTALEZA,1988-05-17,NaN,NaN
1266275,CRATEUS,1970-09-28,NaN,NaN


In [35]:
municipios = df['municipioCaso'].unique().tolist()
municipios_dict = df_municipios.set_index('municipioCaso').to_dict()['regiaoPlanejamentoCaso']
regions = df_municipios['regiaoPlanejamentoCaso'].unique().tolist()

n = 1
for municipio in municipios_dict:
    municipios_dict[municipio] = [municipios_dict[municipio], n]
    n += 1

## Definição do algoritmo


In [36]:
def perturbate(df):
    """
    algoritmo de perturbação.

    df       -- dataframe para ser perturbado
    return   -- dataframe perturbado
    """

    cdf = df.copy()

    def per_date(value):
        """
        pega valores de ano-mês-dia (ex: 1986-04-26)
        pegar valor do ano (ex: 1986)
        dividir pelo dia + mês e arredonda (ex: 1986/26 = 77)
        """
        year = value.year
        month = value.month
        day = value.day

        joker = round(year / (day + month))

        return value + pd.DateOffset(days=joker if month % 2 == 1 else -joker)

    cdf['dataNascimento'] = cdf['dataNascimento'].map(per_date)

    def per_city(value):
        """
        pega 10 cidades aleatóriamente das presentes no dataframe
        compara os pesos das cidades escolhidas com o peso da cidade original
        escolhe a cidade com peso mais próximo da cidade original como novo valor
        """            

        choices = random.choices(municipios, k=10)

        def util_func(original, choices):
            orig_weight = municipios_dict[original][1]
            best_choice = ""
            best_distance = 999

            for choice in choices:
                choice_weight = municipios_dict[choice][1]
                distance = abs(orig_weight - choice_weight)
                if distance < best_distance:
                    best_choice = choice
                    best_distance = distance
            return best_choice

        return util_func(value, choices)

    cdf['municipioCaso'] = cdf['municipioCaso'].map(per_city)

    return cdf


## Aplicação do algoritmo

In [37]:
df

,municipioCaso,dataNascimento,comorbidadeCardiovascularSivep,comorbidadeDiabetesSivep
19,SOBRAL,2003-08-14,NaN,NaN
20,PACAJUS,1983-11-07,NaN,NaN
21,HORIZONTE,1982-01-14,NaN,NaN
25,FORTALEZA,1992-03-12,NaN,NaN
28,CAUCAIA,1970-03-06,NaN,NaN
...,...,...,...,...
1266272,FORTALEZA,1982-12-08,NaN,NaN
1266273,FORTALEZA,1980-12-02,NaN,NaN
1266274,FORTALEZA,1988-05-17,NaN,NaN
1266275,CRATEUS,1970-09-28,NaN,NaN


In [38]:
df_anon = perturbate(df)
df_anon

,municipioCaso,dataNascimento,comorbidadeCardiovascularSivep,comorbidadeDiabetesSivep
19,VARZEA ALEGRE,2003-05-15,NaN,NaN
20,ALCANTARAS,1983-07-20,NaN,NaN
21,TRAIRI,1982-05-26,NaN,NaN
25,PARAIPABA,1992-07-23,NaN,NaN
28,MARACANAU,1970-10-11,NaN,NaN
...,...,...,...,...
1266272,CHOROZINHO,1982-08-31,NaN,NaN
1266273,TRAIRI,1980-07-14,NaN,NaN
1266274,PARACURU,1988-08-15,NaN,NaN
1266275,PARAMOTI,1970-08-06,NaN,NaN


In [39]:
print(df['dataNascimento'].describe(datetime_is_numeric=True))
print("")
print(df_anon['dataNascimento'].describe(datetime_is_numeric=True))

count                           911711
mean     1980-10-13 21:32:24.793909440
min                1691-02-27 00:00:00
25%                1968-11-09 00:00:00
50%                1982-10-19 00:00:00
75%                1994-03-07 00:00:00
max                2021-01-22 00:00:00
Name: dataNascimento, dtype: object

count                           911711
mean     1980-11-13 12:56:59.520440256
min                1691-04-26 00:00:00
25%                1968-10-22 00:00:00
50%                1982-10-06 00:00:00
75%                1994-05-16 00:00:00
max                2023-10-08 00:00:00
Name: dataNascimento, dtype: object


In [40]:
print(df['municipioCaso'].describe())
print("")
print(df_anon['municipioCaso'].describe())

count        911711
unique          184
top       FORTALEZA
freq         238318
Name: municipioCaso, dtype: object

count        911711
unique          184
top       FORTALEZA
freq          18861
Name: municipioCaso, dtype: object


## Gerar CSVs

In [41]:
df.to_csv(filename[:-4] + '_Clean.csv', index=False)
df_anon.to_csv(filename_out, index=False)